# 6.2. Случайный лес

Ваша задача - написать класс `random_forest` для решения задачи классификации на основе датасета Ирисов Фишера (`sklearn.datasets.load_iris`), принимающий на вход конструктора аргументы `n_estimators`, `max_depth`, `subspaces_dim` и `random_state`. описание этих аргументов приведено ниже. У этого класса должны быть определены методы `.fit()` и `.predict()`, а также поле `._estimators`, в котором должен храниться список алгоритмов, используемых в ансамбле.

Для создания обучающей подвыборки для каждого из базовых классификаторов, Вы можете воспользоваться классом `sample`, который Вы реализовали в прошлом задании. В случае его использования, не забудьте включить его описание в файл с Вашим решением текущего задания. Мы также предлагаем вам организовать выбор подпространств для каждого дерева посредством заполнения списка `subspace_idx`, в котором будут логироваться выбранные для каждого базового классификатора подпространства.

Замечание: в рамках выполнения данного задания запрещено использовать класс `sklearn.ensemble.RandomForestClassifier`. Такой код не пройдёт проверку.

Подберите также гиперпараметры, на которых ваш алгоритм получает наилучшее качество (с точки зрения метрики accuracy, доли правильных ответов) на тестовой выборке с параметром `test_size`=0.3, задайте их в виде глобальных переменных N_ESTIMATORS, MAX_DEPTH, SUBSPACE_DIM.

Шаблон класса:

In [1]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
np.random.seed(42)

class sample(object):
    def init(self, X, n_subspace):
        self.idx_subspace = self.random_subspace(X, n_subspace)

    def call(self, X, y):
        idx_obj = self.bootstrap_sample(X)
        X_sampled, y_sampled = self.get_subsample(X, y, self.idx_subspace, idx_obj)
        return X_sampled, y_sampled

    @staticmethod
    def bootstrap_sample(X, random_state=42):
        return np.array(np.unique(np.random.choice(len(X), len(X)))) # len(X) - это число строк- объектов в массиве X
    @staticmethod
    def random_subspace(X, n_subspace, random_state=42):
        return np.array(np.random.choice(len(X[0]), n_subspace, replace=False)) # 1ый аргумент из какого мн-ва берутся числа, 2ой аргумент - сколько чисел берется
    @staticmethod
    def get_subsample(X, y, idx_subspace, idx_obj):
        x_sampled=X[np.ix_(idx_obj, idx_subspace)] # ix_ - берет пересечение индексов
        y_sampled=y[np.array(idx_obj)]
        return np.array(x_sampled), np.array(y_sampled)

class random_forest(object):
    def init(self, n_estimators: int, max_depth: int, subspaces_dim: int, random_state: int):
        self.n_estimators = n_estimators # число деревьев
        self.max_depth = max_depth
        self.subspaces_dim = subspaces_dim # число признаков, которые будем использовать
        self.random_state = random_state
        self.algs = []
        self.cols = [] # массив массивов - то есть список номеров признаков для каждого дерева
    def fit(self, X, y):
        for i in range(self.n_estimators):
            s = sample(X, self.subspaces_dim)
            X_sampled, y_sampled = s.get_subsample(X, y, s.idx_subspace, s.bootstrap_sample(X)) # idx_subspace-число столбцов в выборке, bootstrap_sample-число строк в выборке
            self.cols.append(s.idx_subspace)
            clf = DecisionTreeClassifier(max_depth=self.max_depth, random_state=self.random_state)
            clf.fit(X_sampled, y_sampled)
            self.algs.append(clf)

        self.algs=np.array(self.algs)
        self.cols=np.array(self.cols)
        pass
    def predict(self, X):
        n = np.shape(X)[0]
        y = []
        for i in range(n): # идем по каждому объекту
            lst = [] # на каждый объект пустой список ответов
            count = [0] * 3
            for clf_num in range(self.n_estimators): # идем по деревьям
                tmp_X = [] # обнуляем для кажого дерева свою подвыборку
                for j in range(len(self.cols[clf_num])): # составляем выборку tmp_X cols[clf_num]
                      tmp_X.append(X[i][self.cols[clf_num][j]]) # cols[clf_num]
                pred = self.algs[clf_num].predict(np.array(tmp_X).reshape(1,-1))
                lst.append(pred) # обучаем деревом self.algs[clf_num] на подвыборке tmp_X
            lst = np.array(lst).reshape(1,-1)

            # переделываем из массива [[1,2,3]] в массив [1,2,3]
            x1 = len(lst[0])
            y1 = len(lst)
            lst_new = []
            for q in range(y1):
                for w in lst[q]:
                    lst_new.append(w)
            lst = lst_new

            for num in range(len(lst)): # идем по списку lst, то есть по ответам для каждого объекта i
                count[lst[num]] += 1 # заполняем массив count
            y.append(np.argmax(count)) # кладем в y нужный класс по методу простого голосования
        return np.array(y)


N_ESTIMATORS = 7
MAX_DEPTH = 11
SUBSPACE_DIM = 2